In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import string

engine = create_engine("postgresql://localhost/mimic?user=xxxx&password=xxxx") 
# user name and password needs to be modified based on the local setup of the postgress server  

In [ ]:
total_row = 58929 ## use a few rows to test 
rowlimit = total_row
fname = "dis_t.txt"
gname = "h_t.csv"
max_size = 1500


def smart_truncate(content, length=max_size):
    word = content.split()
    if len(word) <= length:
        return content
    else:
        return ' '.join(word[:length])
    
with open(gname, "a") as g:
    g.write('HADM_ID\n')


df = pd.read_sql_query(
    'select HADM_ID, text from mimic.mimiciii.noteevents where category=\'Discharge summary\'  offset ' + 
    str(rowlimit) + ' limit ''' + str(rowlimit),
    con=engine)  # change limit number
df = df.sort_values('HADM_ID').reset_index()
df1 = df.groupby('HADM_ID').agg(lambda x: ''.join(x.unique()))
df1 = df1.reset_index()

with open(fname, "a") as f, open(gname, "a") as g:
    for index, line in df1.iterrows():
        t = " ".join(line[1].split())
        text = "".join([w for w in t if w not in string.punctuation])
        t2 = text.lower()
        ttrial = t2.split()
        tt = [y for y in ttrial if not y.isnumeric()]
        tfinal = " ".join(tt)
        ttrun = smart_truncate(tfinal)
        f.write(str(tfinal) + '\n')
        g.write(str(line[0]) + '\n')
    